<a href="https://colab.research.google.com/github/Larai15/kubernetes/blob/master/Copy_of_INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [0]:
# Write your code here

import nltk
import pandas 
import textblob
from tabulate import tabulate

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

#1)
result=[]
with open("imdb.csv") as f:
    for line in f:
        token=line.split()
        print(token)
        trigrams = nltk.trigrams(token)
        fdist = nltk.FreqDist(trigrams)
        for k,v in fdist.items():
            result.append([k,v])
            # print(k,v)

print(tabulate(result,headers=["Trigrams","Frequency"]))

#2)
def bigramCalculator(file):

    lst = [] # This will contain the tokens
    unigrams = {} # for unigrams and their counts
    bigrams = {} # for bigrams and their counts

    text = open(file, 'r').read()
    lst = text.strip().split()
    print('Read ', len(lst), ' tokens...')

    for l in lst:
        if not l in unigrams:
            unigrams[l] = 1
        else:
            unigrams[l] += 1

    print('Generated ', len(unigrams), ' unigrams...') 

    # 3. Generate bigrams with frequencies
    for i in range(len(lst) - 1):
        temp = (lst[i], lst[i+1]) # Tuples are easier to reuse than nested lists
        if not temp in bigrams:
            bigrams[temp] = 1
        else:
            bigrams[temp] += 1

    print('Generated ', len(bigrams), ' bigrams...')

    total_corpus = sum(unigrams.values())


    print('Calculating bigram probabilities')

    result = []
    for k,v in bigrams.items():
        first_word = k[0]
        first_word_count = unigrams[first_word]
        bi_prob = bigrams[k] / unigrams[first_word]
        result.append([*k,bi_prob])
    print(tabulate(result, headers=['First Word',"Second World","Probability"]))


# Callings
bigramCalculator('imdb.csv')

#3)
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import RegexpParser
from nltk import Tree
import pandas as pd

result = []
def get_continuous_chunks(text, chunk_func=ne_chunk):
    chunked = chunk_func(pos_tag(word_tokenize(text)))
    continuous_chunk = []
    current_chunk = []

    for subtree in chunked:
        if type(subtree) == Tree:
            current_chunk.append(" ".join([token for token, pos in subtree.leaves()]))
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
        else:
            continue
    
    result.append(continuous_chunk)
    return continuous_chunk
data_list = []
with open("imdb.csv", "r") as fp:
    data_list = fp.readlines()

df = pd.DataFrame({'text':data_list})

df['text'].apply(lambda sent: get_continuous_chunks((sent)))
flat_list = [item for sublist in result for item in sublist]
print(flat_list)





[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
['USER,REVIEW']
['JF500,', 'As', 'a', 'viewer', 'that', 'actually', 'went', 'to', 'TIFF', 'and', 'witnessed', 'this', 'film', 'and', "didn't", 'want', 'to', 'believe', 'the', 'hype,', 'it', 'is', 'an', 'absolute', 'MASTERPIECE', 'and', 'Phoenix', 'is', 'a', 'certified', 'legend.']
[]
['MihaVrhunc,', 'Outstanding', 'movie', 'with', 'a', 'haunting', 'performance', 'and', 'best', 'character', 'development', 'ever', 'seen']
[]
['lesterarno

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [0]:
# Write your code here

import nltk
import numpy as np
import random
import string

import bs4 as bs
import urllib.request
import re
from tabulate import tabulate
from scipy import spatial
nltk.download('punkt')


article_text = ''

with open("imdb.csv") as f:
    article_text = f.read().replace('\n', '')

corpus = nltk.sent_tokenize(article_text)

for i in range(len(corpus )):
    corpus [i] = corpus [i].lower()
    corpus [i] = re.sub(r'\W',' ',corpus [i])
    corpus [i] = re.sub(r'\s+',' ',corpus [i])

wordfreq = {}
for sentence in corpus:
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1

import heapq
most_freq = heapq.nlargest(200, wordfreq, key=wordfreq.get)

word_idf_values = {}
for token in most_freq:
    doc_containing_word = 0
    for document in corpus:
        if token in nltk.word_tokenize(document):
            doc_containing_word += 1
    word_idf_values[token] = np.log(len(corpus)/(1 + doc_containing_word))

word_tf_values = {}
for token in most_freq:
    sent_tf_vector = []
    for document in corpus:
        doc_freq = 0
        for word in nltk.word_tokenize(document):
            if token == word:
                  doc_freq += 1
        word_tf = doc_freq/len(nltk.word_tokenize(document))
        sent_tf_vector.append(word_tf)
    word_tf_values[token] = sent_tf_vector

tfidf_values = []
for token in word_tf_values.keys():
    tfidf_sentences = []
    for tf_sentence in word_tf_values[token]:
        tf_idf_score = tf_sentence * word_idf_values[token]
        tf_idf_score = float(round(tf_idf_score, 3))
        tfidf_sentences.append(tf_idf_score)
    tfidf_values.append([token] + tfidf_sentences)
print(tabulate(tfidf_values))

print("\nCOSINE\n")
data_cosine_tokens = ['']
data_cosine = [data_cosine_tokens]
for data1 in tfidf_values:
    data_weight1 = data1[1:]
    cosine  = [data1[0]]
    data_cosine_tokens.append(data1[0])
    for data2 in tfidf_values:
       data_weight2 = data2[1:]
       result = 1 - spatial.distance.cosine(data_weight1, data_weight2)
       cosine.append(result)
    data_cosine.append(cosine)
print(tabulate(data_cosine))



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
----------------------  ------  ------
a                       -0.019  -0.033
and                     -0.025  -0.004
to                      -0.013  -0.007
the                     -0.013  -0.004
is                       0       0
masterpiece             -0.013  -0.004
best                    -0.006  -0.007
joker                    0       0
of                       0       0
as                      -0.006  -0.004
went                    -0.006  -0.004
this                    -0.006  -0.004
t                       -0.006  -0.004
hype                     0       0
it                      -0.006  -0.004
an                      -0.006  -0.004
movie                   -0.006  -0.004
ever                    -0.006  -0.004
for                      0       0
than                     0       0
oscar                    0       0
dark                     0       0
real                    

/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


----------------------  ------------------  ------------------  ------------------  ------------------  ---  ------------------  ------------------  -----  ---  ------------------  ------------------  ------------------  ------------------  ----  ------------------  ------------------  ------------------  ------------------  ---  ----  -----  ----  ----  ---  ---  ----  -----------  ------  ----  --------  ----  ---------  ----  ----  ----  -------  --------  -------  ---------  ------  ----------  -----------  ----  --------  -----------  ---------  -----------  ---------------------  ----  -------  ------  ---  ----------------  -------  ---  -----  ------  -----------  ---------------  ------------  ----  -------  ---  ----  -----  ------------  ------  ----  ------  ---------  -------------  -----  ------  ---------  -------------  -----  -------  --------  --------  ------  ------  -----  -------  ------  ---  ----------------  -----  -------  -------------  ----  --------------- 

# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [0]:
# The GitHub link of your final csv file

# Link: https://github.com/Larai15/kubernetes/blob/master/imdb_sentiment.csv